In [4]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv, get_key


load_dotenv()
GEMINI_API_KEY = get_key(dotenv_path=".env", key_to_get="GEMINI_API_KEY")


llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=GEMINI_API_KEY)


messages = [
    ("system", """You are a mental health assistnant.

You will be given a conversation and need to classify the latest user message into one of the following categories:

Normal                  
Depression              
Suicidal                
Anxiety                  
Bipolar                  
Stress                   
Personality disorder

Give the response in a json format with the following keys:

{
    "category": "<category>"
}
Strictly follow the format and do not add any additional text or explanation.
"""),
    ("user", "What is the capital of France?")]

response = llm.invoke(messages)
print(response.content)

```json
{
    "category": "Normal"
}
```


In [ ]:
# load csv
import pandas as pd
import os
import json

df = pd.read_csv("Subset Data.csv")

# loop through all the rows to get the response based on the staement column
for index, row in df.iterrows():
    messages = [
    ("system", """You are a mental health assistnant.

You will be given a conversation and need to classify the latest user message into one of the following categories:

Normal                  
Depression              
Suicidal                
Anxiety                  
Bipolar                  
Stress                   
Personality disorder

Give the response in a json format with the following keys:

{
    "category": "<category>"
}
Strictly follow the format and do not add any additional text or explanation.
""")]
    try:
        query = row["statement"]
        print(f"Row {index} query: {query}")
        messages.append(("user", query))

        response = llm.invoke(messages)
        response = response.content
        # strip the json response to get the category
        response = response.split('\n')[1:-1]
        # join the lines back together
        response = '\n'.join(response)
        response_json = json.loads(response)

        df.at[index, "category"] = response_json["category"]
        print(f"Row {index}: {response_json['category']}")

    except Exception as e:
        print(f"Row {index} reponse: {e}")
        df.at[index, "category"] = "Error"
        continue
    # save the dataframe to a csv file

df.to_csv("Subset Data only LLM responses.csv", index=False)

Row 0: All this work, all this pressure that everyone puts on you to succeed. To go to a good college, get a good job, the normal things a lot of parents ask. All for what? I work my entire life and then what? Am I supposed to enjoy my ofttimes from studying or working being an ugly, socially awkward loser? Not able to talk to anyone, have friends; even when doing normally enjoyed things (video games, time off, etc.) all I can think about is how everyone else is probably enjoying their time with other people. Am I working for something or am I just working for the sake of working just because everyone tells me that is what I am supposed to do. Do people only tell you it gets better for the sake of having one more wage slave satisfy their demand for goods? What is this all for? All for what?
Row 0: Depression
Row 1: I am tired of living in a world full of judgment and killing each other I am tired of my family criticizing me and my sisters weight every time i get them in trouble uninten

KeyboardInterrupt: 